# Pytorch 快速上手

以 FashionMNIST 服饰分类模型为例

In [20]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [21]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [22]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [23]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [25]:
# noinspection PyShadowingNames
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        # Compute prediction error
        pred = model(x)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            
# noinspection PyShadowingNames
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # enable eval mode
    model.eval()
    test_loss, correct = 0, 0
    # freeze gradient descent
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [26]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309470  [   64/60000]
loss: 2.299921  [ 6464/60000]
loss: 2.278701  [12864/60000]
loss: 2.270685  [19264/60000]
loss: 2.252063  [25664/60000]
loss: 2.220900  [32064/60000]
loss: 2.233458  [38464/60000]
loss: 2.201101  [44864/60000]
loss: 2.205760  [51264/60000]
loss: 2.170005  [57664/60000]
Test Error: 
 Accuracy: 40.4%, Avg loss: 2.163715 

Epoch 2
-------------------------------
loss: 2.175881  [   64/60000]
loss: 2.167834  [ 6464/60000]
loss: 2.114469  [12864/60000]
loss: 2.129708  [19264/60000]
loss: 2.074502  [25664/60000]
loss: 2.018675  [32064/60000]
loss: 2.047115  [38464/60000]
loss: 1.974840  [44864/60000]
loss: 1.989403  [51264/60000]
loss: 1.917152  [57664/60000]
Test Error: 
 Accuracy: 55.2%, Avg loss: 1.911286 

Epoch 3
-------------------------------
loss: 1.941716  [   64/60000]
loss: 1.913897  [ 6464/60000]
loss: 1.803011  [12864/60000]
loss: 1.845849  [19264/60000]
loss: 1.728858  [25664/60000]
loss: 1.682594  [32064/600

## 保存整个模型

保存整个模型意味着将模型的结构和参数一起保存到一个文件中。这种方法简单直观，但缺点是保存的文件可能会包含一些不必要的信息（如优化器状态），并且加载模型时需要有相同的模型类定义
 
```python
# 保存模型
torch.save(model, 'model.pth')

# 加载模型
model = torch.load('model.pth')
model.eval()  # 设置为评估模式
```

## 保存状态字典
    
状态字典包含了模型的参数，不包含模型的结构。这种方法更加灵活，可以轻松地在不同的模型结构之间迁移参数

```python
# 保存模型的状态字典
torch.save(model.state_dict(), 'model_state_dict.pth')

# 加载模型的状态字典
model = MyModel()  # 需要先实例化模型
model.load_state_dict(torch.load('model_state_dict.pth'))
model.eval()  # 设置为评估模式
```
  
## 加载模型

要加载模型权重，需要先创建相同模型的实例，然后使用 load_state_dict() 方法加载参数，加载模型时可以指定设备，以确保模型参数在正确的设备上

```python
model.load_state_dict(torch.load('model_state_dict.pth', map_location=torch.device('cpu')))
```

>PyTorch 模型将学习到的参数存储在一个名为 state_dict 的内部状态字典中。可以通过 torch.save 方法持久化这些参数

In [27]:
torch.save(model.state_dict(), "./model/fashion_mnist.pth")
print("Saved PyTorch Model State to ./model/fashion_mnist.pth")

Saved PyTorch Model State to ./model/fashion_mnist.pth


In [28]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("./model/fashion_mnist.pth", weights_only=True))

<All keys matched successfully>

In [31]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model.forward(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Output tensor: {pred[0]}')
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Output tensor: tensor([-3.5301, -5.3487, -2.3215, -2.2501, -1.6678,  4.6115, -2.0385,  4.2946,
         2.4023,  4.9914])
Predicted: "Ankle boot", Actual: "Ankle boot"


>请确保在推理之前调用 model.eval() 方法，停止梯度计算降低内存占用，同时将 dropout 和批归一化层设置为评估模式，否则将导致不一致的推理结果